# Exercise 8
Practice training a deep neural network on the CIFAR10 image dataset:

a. Build a DNN with 20 hidden layers of 100 neurons each (that’s too many, but it’s the point of this exercise). Use He initialization and the ELU activation
function.

b. Using Nadam optimization and early stopping, train the network on the CIFAR10 dataset. You can load it with keras.datasets.cifar10.load_data(). The dataset is composed of 60,000 32 × 32–pixel color images (50,000 for training, 10,000 for testing) with 10 classes, so you’ll need a softmax output
layer with 10 neurons. Remember to search for the right learning rate each time you change the model’s architecture or hyperparameters.

c. Now try adding Batch Normalization and compare the learning curves: Is it converging faster than before? Does it produce a better model? How does it
affect training speed?

d. Try replacing Batch Normalization with SELU, and make the necessary adjustements to ensure the network self-normalizes (i.e., standardize the input features,
use LeCun normal initialization, make sure the DNN contains only a sequence of dense layers, etc.).

e. Try regularizing the model with alpha dropout. Then, without retraining your model, see if you can achieve better accuracy using MC Dropout.

f. Retrain your model using 1cycle scheduling and see if it improves training speed and model accuracy.

In [1]:
import tensorflow as tf
from tensorflow import keras
from functools import partial

# Since we're creating a sequential model and want to keep our code neat and clean I'm going to create a partial object
# and add 20 DenseLayers using a for loop to make life easier and faster
DenseLayer = partial(keras.layers.Dense,
                     activation="elu",
                     kernel_initializer="he_normal")
# initialize the model
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[32, 32, 3]),])
# add 20 hidden layers of 100 neuron Dense layers 
for _ in range(0,20):
    model.add(DenseLayer(100))
# add our output layer
model.add(keras.layers.Dense(10, activation="softmax"))
# compile the model
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.Nadam(lr=5e-5),
              metrics=["accuracy"])

import os
run_index = 1 # increment this at every run
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_{:03d}".format(run_index))
run_logdir

checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_model.h5", save_best_only=True)

In [12]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 3072)              0         
_________________________________________________________________
dense_21 (Dense)             (None, 100)               307300    
_________________________________________________________________
dense_22 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_23 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_24 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_25 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_26 (Dense)             (None, 100)              

In [14]:
# compile the model
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.Nadam(lr=1e-3),
              metrics=["accuracy"])
# create an early stopping callback
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

In [2]:
# Now that the model is built, let's download our data
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=10000, random_state=42)

In [34]:
# train the model
history = model.fit(X_train, y_train, epochs=100,
                    validation_data=(X_valid, y_valid),
                    callbacks=[early_stopping_cb, checkpoint_cb])

Train on 40000 samples, validate on 10000 samples
Epoch 1/100
40000/40000 [==============================] - 17s 429us/sample - loss: 4.3715 - accuracy: 0.1601 - val_loss: 2.1827 - val_accuracy: 0.1908
Epoch 2/100
40000/40000 [==============================] - 14s 342us/sample - loss: 2.0893 - accuracy: 0.2342 - val_loss: 2.0220 - val_accuracy: 0.2546
Epoch 3/100
40000/40000 [==============================] - 14s 361us/sample - loss: 1.9651 - accuracy: 0.2803 - val_loss: 1.9471 - val_accuracy: 0.2951
Epoch 4/100
40000/40000 [==============================] - 15s 364us/sample - loss: 1.8880 - accuracy: 0.3143 - val_loss: 1.8615 - val_accuracy: 0.3247
Epoch 5/100
40000/40000 [==============================] - 14s 360us/sample - loss: 1.8287 - accuracy: 0.3356 - val_loss: 1.8118 - val_accuracy: 0.3423
Epoch 6/100
40000/40000 [==============================] - 14s 351us/sample - loss: 1.7768 - accuracy: 0.3535 - val_loss: 1.7557 - val_accuracy: 0.3631
Epoch 7/100
40000/40000 [=============

In [36]:
model = keras.models.load_model("my_cifar10_model.h5")
model.evaluate(X_valid, y_valid)

10000/10000 [==============================] - 1s 98us/sample - loss: 1.5224 - accuracy: 0.4654


[1.5223829399108886, 0.4654]

In [8]:
import numpy as np
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [38]:
DenseLayer = partial(keras.layers.Dense,
                     activation="elu",
                     kernel_initializer="he_normal")
# initialize the model
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[32, 32, 3]),
    keras.layers.BatchNormalization(),
])

# add 20 hidden layers of 100 neuron Dense layers 
for _ in range(0,20):
    model.add(DenseLayer(100))
    model.add(keras.layers.BatchNormalization())
# add our output layer
model.add(keras.layers.Dense(10, activation="softmax"))
# compile the model
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.Nadam(lr=5e-4),
              metrics=["accuracy"])

# callbacks
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_bn_model.h5", save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_bn_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

# train the model
model.fit(X_train, y_train, epochs=100,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)
# test the model
model = keras.models.load_model("my_cifar10_bn_model.h5")
model.evaluate(X_valid, y_valid)

Train on 40000 samples, validate on 10000 samples
Epoch 1/100
40000/40000 [==============================] - 35s 878us/sample - loss: 1.9428 - accuracy: 0.3000 - val_loss: 1.7137 - val_accuracy: 0.3789
Epoch 2/100
40000/40000 [==============================] - 23s 583us/sample - loss: 1.7338 - accuracy: 0.3850 - val_loss: 1.6100 - val_accuracy: 0.4244
Epoch 3/100
40000/40000 [==============================] - 24s 595us/sample - loss: 1.6523 - accuracy: 0.4124 - val_loss: 1.5677 - val_accuracy: 0.4505
Epoch 4/100
40000/40000 [==============================] - 24s 595us/sample - loss: 1.6033 - accuracy: 0.4315 - val_loss: 1.5255 - val_accuracy: 0.4518
Epoch 5/100
40000/40000 [==============================] - 24s 596us/sample - loss: 1.5522 - accuracy: 0.4486 - val_loss: 1.5028 - val_accuracy: 0.4660
Epoch 6/100
40000/40000 [==============================] - 24s 600us/sample - loss: 1.5107 - accuracy: 0.4655 - val_loss: 1.4669 - val_accuracy: 0.4740
Epoch 7/100
40000/40000 [=============

[1.3712681060791017, 0.525]

# Part D

In [9]:
# initialize the model
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[32, 32, 3]),
    keras.layers.BatchNormalization(),
])

# add 20 hidden layers of 100 neuron Dense layers 
for _ in range(0,20):
    model.add(keras.layers.Dense(100,activation="selu",kernel_initializer="lecun_normal"))
# add our output layer
model.add(keras.layers.Dense(10, activation="softmax"))

# compile the model
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.Nadam(lr=7e-5),
              metrics=["accuracy"])

# callbacks
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_selu_model.h5", save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_bn_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

In [10]:
# train the model
model.fit(X_train, y_train, epochs=100,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)
# test the model
model = keras.models.load_model("my_cifar10_selu_model.h5")
model.evaluate(X_valid, y_valid)

Train on 40000 samples, validate on 10000 samples
Epoch 1/100
40000/40000 [==============================] - 20s 495us/sample - loss: 1.8818 - accuracy: 0.3294 - val_loss: 1.7372 - val_accuracy: 0.3807
Epoch 2/100
40000/40000 [==============================] - 15s 381us/sample - loss: 1.6861 - accuracy: 0.3986 - val_loss: 1.6407 - val_accuracy: 0.4107
Epoch 3/100
40000/40000 [==============================] - 16s 398us/sample - loss: 1.5886 - accuracy: 0.4327 - val_loss: 1.5976 - val_accuracy: 0.4305
Epoch 4/100
40000/40000 [==============================] - 16s 389us/sample - loss: 1.5304 - accuracy: 0.4547 - val_loss: 1.5554 - val_accuracy: 0.4496
Epoch 5/100
40000/40000 [==============================] - 15s 385us/sample - loss: 1.4782 - accuracy: 0.4756 - val_loss: 1.5455 - val_accuracy: 0.4518
Epoch 6/100
40000/40000 [==============================] - 16s 392us/sample - loss: 1.4388 - accuracy: 0.4873 - val_loss: 1.5285 - val_accuracy: 0.4607
Epoch 7/100
40000/40000 [=============

[1.4512366218566894, 0.491]

# Part E

In [18]:
# reset everything
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [15]:
# standardizing the data for input
pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)
X_train_scaled = (X_train - pixel_means) / pixel_stds
X_valid_scaled = (X_valid - pixel_means) / pixel_stds
X_test_scaled = (X_test - pixel_means) / pixel_stds

In [20]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[32, 32, 3]),
    keras.layers.AlphaDropout(rate=0.2)
])

# add 20 hidden layers of 100 neuron Dense layers 
for _ in range(0,20):
    model.add(keras.layers.Dense(100,activation="selu",kernel_initializer="lecun_normal"))

# add the dropout layer
model.add(keras.layers.AlphaDropout(rate=0.2))
# add our output layer
model.add(keras.layers.Dense(10, activation="softmax"))

# compile the model
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.Nadam(lr=5e-4),
              metrics=["accuracy"])

# callbacks
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_alphadrop_model.h5", save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_bn_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

In [21]:
# train the model
model.fit(X_train_scaled, y_train, epochs=100,
          validation_data=(X_valid_scaled, y_valid),
          callbacks=callbacks)
# test the model
model = keras.models.load_model("my_cifar10_alphadrop_model.h5")
model.evaluate(X_test_scaled, y_valid)

Train on 40000 samples, validate on 10000 samples
Epoch 1/100
40000/40000 [==============================] - 20s 493us/sample - loss: 1.9650 - accuracy: 0.2997 - val_loss: 1.8046 - val_accuracy: 0.3535
Epoch 2/100
40000/40000 [==============================] - 15s 384us/sample - loss: 1.7608 - accuracy: 0.3765 - val_loss: 1.7762 - val_accuracy: 0.3936
Epoch 3/100
40000/40000 [==============================] - 15s 384us/sample - loss: 1.6817 - accuracy: 0.4079 - val_loss: 1.7729 - val_accuracy: 0.4008
Epoch 4/100
40000/40000 [==============================] - 15s 385us/sample - loss: 1.6235 - accuracy: 0.4289 - val_loss: 1.6615 - val_accuracy: 0.4321
Epoch 5/100
40000/40000 [==============================] - 15s 387us/sample - loss: 1.5776 - accuracy: 0.4464 - val_loss: 1.6576 - val_accuracy: 0.4554
Epoch 6/100
40000/40000 [==============================] - 15s 385us/sample - loss: 1.5393 - accuracy: 0.4638 - val_loss: 1.7310 - val_accuracy: 0.4398
Epoch 7/100
40000/40000 [=============

[4.934977342224121, 0.0999]

In [24]:
model.evaluate(X_test_scaled, y_test)

10000/10000 [==============================] - 1s 65us/sample - loss: 1.6249 - accuracy: 0.4781


[1.6249331756591796, 0.4781]